### **Advanced SQL**

<font color="red">File access required:</font> In Colab this notebook requires first uploading files **Cities.csv**, **Countries.csv**, **Players.csv**, and **Teams.csv** using the *Files* feature in the left toolbar. If running the notebook on a local computer, simply ensure these files are in the same workspace as the notebook.

In [ ]:
!pip install prettytable==0.7.2
!pip install ipython-sql

In [ ]:
# Set-up
%load_ext sql
%sql sqlite://
import pandas as pd

In [ ]:
# Create database tables from CSV files
with open('Cities.csv') as f: Cities = pd.read_csv(f, index_col=0)
%sql drop table if exists Cities;
%sql --persist Cities
with open('Countries.csv') as f: Countries = pd.read_csv(f, index_col=0)
%sql drop table if exists Countries;
%sql --persist Countries

#### Look at sample of Cities and Countries tables

In [ ]:
%%sql
select * from Cities limit 5

In [ ]:
%%sql
select * from Countries limit 5

### Duplicates, table variables

*Warm-up: Find all cities in the EU with temperature > 15*

In [ ]:
%%sql
select city
from Cities, Countries
where Cities.country = Countries.country
and EU = 'yes' and temperature > 15

*Modify previous query to return EU countries that have a city with temperature > 15, remove duplicates*

*Find number of countries that have a city with latitude > 60 (start with country list then fix)*

In [ ]:
%%sql
select country
from Cities
where latitude > 60

*Modify first query to use table variables*

In [ ]:
%%sql
select city
from Cities, Countries
where Cities.country = Countries.country
and EU = 'yes' and temperature > 15

*Find all pairs of cities with the same longitude; return the city pairs and their (shared) longitude - notice what's wrong and fix it*

In [ ]:
%%sql
select C1.city, C2.city, C1.longitude
from Cities C1, Cities C2
where C1.longitude = C2.longitude

*Find all pairs of cities that are near each other, i.e., longitude and latitude are both less than 0.5 apart; return city pairs*

In [ ]:
%%sql
select C1.city, C2.city
from Cities C1, Cities C2
where abs(C1.longitude - C2.longitude) < .5
and abs(C1.latitude - C2.latitude) < .5
and C1.city < C2.city

### <font color = 'green'>**Your Turn**</font>

*Find all pairs of cities with the same temperature. Return the city pairs along with their shared temperature. What do you think about the data after seeing the answer?*

In [3]:
import pandas as pd
import sqlite3

def setup_db():
    conn = sqlite3.connect(':memory:')
    files = {
        'Cities': 'Cities.csv',
        'Countries': 'Countries.csv',
        'Players': 'Players.csv',
        'Teams': 'Teams.csv',
        'Titanic': 'Titanic.csv'
    }
    for table, file in files.items():
        try:
            pd.read_csv(file).to_sql(table, conn, index=False, if_exists='replace')
        except:
            print(f"Warning: could not load {file}")
    return conn

conn = setup_db()

def run(query, title):
    print(f"\n--- {title} ---")
    try:
        res = pd.read_sql_query(query, conn)
        print(res.to_markdown(index=False))
    except Exception as e:
        print(f"Error: {e}")

run("""
SELECT C1.city, C2.city, C1.temperature
FROM Cities C1, Cities C2
WHERE C1.temperature = C2.temperature AND C1.city < C2.city;
""", "World 1: Pairs of cities with same temperature")


--- World 1: Pairs of cities with same temperature ---
| city       | city         |   temperature |
|:-----------|:-------------|--------------:|
| Andorra    | Sarajevo     |          9.6  |
| Augsburg   | Innsbruck    |          4.54 |
| Baia Mare  | Debrecen     |          8.87 |
| Balti      | Botosani     |          8.23 |
| Basel      | Elblag       |          6.68 |
| Basel      | Freiburg     |          6.68 |
| Basel      | Mulhouse     |          6.68 |
| Basel      | Zurich       |          6.68 |
| Bialystok  | Hrodna       |          6.07 |
| Bonn       | Maastricht   |          8.63 |
| Bratislava | Brugge       |          9.65 |
| Bratislava | Gyor         |          9.65 |
| Bratislava | Lille        |          9.65 |
| Brno       | Vienna       |          7.86 |
| Brugge     | Gyor         |          9.65 |
| Brugge     | Lille        |          9.65 |
| Bucharest  | Le Mans      |         10.59 |
| Bucharest  | Ruse         |         10.59 |
| Bytom      | Ostrava  

### Subqueries in Where clause

*Find all countries in the Countries table with no city in the Cities table*

In [ ]:
%%sql
select country
from Countries
where not exists (

  select * from Cities
  where Cities.country = Countries.country

)

*Find countries in the EU that have a city with temperature > 15*

In [ ]:
%%sql
select country
from Countries
where EU = 'yes'
and exists (select * from Cities
            where Cities.country = Countries.country
            and temperature > 15)

*Find number of countries that have a city with latitude > 60 (start with country list)*

In [ ]:
%%sql
select country
from Countries
where exists (select * from Cities
              where Cities.country = Countries.country
              and latitude > 60)

*Find the westernmost city; return the city and longitude*

In [ ]:
%%sql
select city, longitude
from Cities C1
where not exists (select * from Cities C2
                  where C2.longitude < C1.longitude)

*Add easternmost to previous query*

*Westernmost city query using = and min*

In [ ]:
%%sql
select city, longitude
from Cities
where longitude = (select min(longitude) from Cities)

*Find all cities whose temperature is more than 50% higher than the average; return the city, country, and temperature, ordered by descending temperature*

In [ ]:
%%sql
select city, country, temperature
from Cities
where temperature > (select avg(temperature) * 1.5 from Cities)
order by temperature desc

*Number of cities in the EU*

In [ ]:
%%sql
select count()
from Cities
where country in (select country from Countries where EU = 'yes')

*Modify previous query to use "not in"*

*Same query using join instead of subquery*

In [ ]:
%%sql
select count()
from Cities, Countries
where Cities.country = Countries.country
and EU = 'yes'

*Number of countries with no coastline and a city with longitude < 20*

In [ ]:
%%sql
select count()
from Countries
where coastline = 'no'
and exists (select * from Cities where country = Countries.country
            and longitude < 20)

*Same query using join instead of subquery (see what's wrong and fix it)*

In [ ]:
%%sql
select count()
from Countries, Cities
where Countries.country = Cities.country
and coastline = 'no' and longitude < 20

*Find countries in Countries table with no city in Cities table using join instead of subquery (subquery version repeated first)*

In [ ]:
%%sql
select country
from Countries
where not exists (select * from Cities
                  where Cities.country = Countries.country)

In [ ]:
%%sql
FILL IN

### <font color = 'green'>**Your Turn**</font>

*Find all cities in a country whose population is < 2; return the city and country. First write the query without a subquery.*

In [6]:
import pandas as pd
import sqlite3

# Load the data
cities_df = pd.read_csv('Cities.csv')
countries_df = pd.read_csv('Countries.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cities_df.to_sql('Cities', conn, index=False)
countries_df.to_sql('Countries', conn, index=False)

# Define the query without a subquery
query = """
SELECT city, Cities.country
FROM Cities
JOIN Countries ON Cities.country = Countries.country
WHERE population < 2;
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| city       | country    |
|:-----------|:-----------|
| Andorra    | Andorra    |
| Tallinn    | Estonia    |
| Tartu      | Estonia    |
| Daugavpils | Latvia     |
| Riga       | Latvia     |
| Podgorica  | Montenegro |


*Now write the same query using a subquery instead of a join.*

In [7]:
import pandas as pd
import sqlite3

# Load the data
cities_df = pd.read_csv('Cities.csv')
countries_df = pd.read_csv('Countries.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cities_df.to_sql('Cities', conn, index=False)
countries_df.to_sql('Countries', conn, index=False)

# Define the query with a subquery
query = """
SELECT city, country
FROM Cities
WHERE country IN (SELECT country FROM Countries WHERE population < 2);
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| city       | country    |
|:-----------|:-----------|
| Andorra    | Andorra    |
| Daugavpils | Latvia     |
| Podgorica  | Montenegro |
| Riga       | Latvia     |
| Tallinn    | Estonia    |
| Tartu      | Estonia    |


*Find all countries with no city having a temperature > 6*

In [8]:
import pandas as pd
import sqlite3

# Load the data
cities_df = pd.read_csv('Cities.csv')
countries_df = pd.read_csv('Countries.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cities_df.to_sql('Cities', conn, index=False)
countries_df.to_sql('Countries', conn, index=False)

# Define the query to find countries with no city having temp > 6
query = """
SELECT country
FROM Countries
WHERE country NOT IN (
    SELECT country
    FROM Cities
    WHERE temperature > 6
);
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| country       |
|:--------------|
| Cyprus        |
| Estonia       |
| Finland       |
| Iceland       |
| Kosovo        |
| Latvia        |
| Liechtenstein |
| Luxembourg    |
| Norway        |


*Now try to write the same query without a subquery; can you?*

In [9]:
import pandas as pd
import sqlite3

# Load the data
cities_df = pd.read_csv('Cities.csv')
countries_df = pd.read_csv('Countries.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cities_df.to_sql('Cities', conn, index=False)
countries_df.to_sql('Countries', conn, index=False)

# Define the query without a subquery (using LEFT JOIN)
query = """
SELECT Countries.country
FROM Countries
LEFT JOIN Cities ON Countries.country = Cities.country AND Cities.temperature > 6
WHERE Cities.city IS NULL;
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| country       |
|:--------------|
| Cyprus        |
| Estonia       |
| Finland       |
| Iceland       |
| Kosovo        |
| Latvia        |
| Liechtenstein |
| Luxembourg    |
| Norway        |


### Aggregation with Having clause

*Find all countries with average city temperature > 10; return country and average temperature*

In [ ]:
%%sql
select country, avg(temperature)
from Cities
group by country
having avg(temperature) > 10

*Find all countries with more than 5 cities above latitude 50*

In [ ]:
%%sql
select country
from Cities
where latitude > 50
group by country
having count() > 5

*Same query without Having clause*

In [ ]:
%%sql
select distinct country
from Cities C1
where 5 < (select count() from Cities C2
           where C1.country=C2.country
           and latitude > 50)

*Which combinations of EU versus non-EU and coastline versus no-coastline have a minimum population greater than 0.5?*

In [ ]:
%%sql
select EU, coastline, min(population)
from Countries
group by EU, coastline
having min(population) > 0.5

*Find all countries with average city temperature more than 50% higher than the overall average; return country and average temperature*

In [ ]:
%%sql
select country, avg(temperature)
from Cities
group by country
having avg(temperature) > (select 1.5 * avg(temperature) from Cities)

### <font color = 'green'>**Your Turn**</font>

*Find all countries whose average city longitude is lower than the overall average longitude, and whose average city latitude is higher than the overall average latitude. Return the countries. Note: Yes, you can use "and" in Having clauses!*

In [10]:
import pandas as pd
import sqlite3

# Load the data
cities_df = pd.read_csv('Cities.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cities_df.to_sql('Cities', conn, index=False)

# Define the query
query = """
SELECT country
FROM Cities
GROUP BY country
HAVING AVG(longitude) < (SELECT AVG(longitude) FROM Cities)
   AND AVG(latitude) > (SELECT AVG(latitude) FROM Cities);
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| country        |
|:---------------|
| Austria        |
| Belgium        |
| Denmark        |
| Germany        |
| Ireland        |
| Netherlands    |
| Norway         |
| United Kingdom |


### Subqueries in From and Select clauses

*Find all countries with both cold and warm cities -- at least one city with temperature < 9 and one city with temperature > 14*

In [ ]:
%%sql
select distinct C1.country
from Cities C1, Cities C2
where C1.country = C2.country
and C1.temperature < 9 and C2.temperature > 14

*Modify query to also return count of cold and warm cities (then show without column renaming)*

In [ ]:
# (select count() from Cities where country = C1.country and temperature < 9) as numcold,
# (select count() from Cities where country = C1.country and temperature > 14) as numwarm

*Same query using subquery in From clause instead of Select clause*

In [ ]:
%%sql
select Cold.country, numcold, numwarm
from (select country, count() as numcold from Cities
      where temperature < 9 group by country) Cold,
     (select country, count() as numwarm from Cities
      where temperature > 14 group by country) Warm
where Cold.country = Warm.country

### Data modification

*Increase all city temperatures by 10%*

In [ ]:
%%sql
update Cities
set temperature = 1.1 * temperature

In [ ]:
%%sql
select avg(temperature) from Cities

*Increase temperatures another 10% for cities in countries with coastline*

In [ ]:
%%sql
update Cities
set temperature = 1.1 * temperature
where country in (select country from Countries
                  where coastline = 'yes')

*Delete all cities in Turkey*

In [ ]:
%%sql
delete from Cities
where country = 'Turkey'

*Create a new table NonEU containing list of cities (with country) not in the EU*

In [ ]:
%%sql
drop table if exists NonEU;
create table NonEU(city, country);
insert into NonEU
  select city, country from cities
  where country in (select country from Countries
                    where EU = 'no');
select * from NonEU

*Add your city*

In [ ]:
%%sql
insert into NonEU values ('my-city','my-country');
select * from NonEU

### <font color = 'green'>**Your Turn - Advanced SQL on World Cup Data**</font>

In [ ]:
# Create database tables from CSV files
with open('Players.csv') as f: Players = pd.read_csv(f, index_col=0)
%sql drop table if exists Players;
%sql --persist Players
with open('Teams.csv') as f: Teams = pd.read_csv(f, index_col=0)
%sql drop table if exists Teams;
%sql --persist Teams

#### Look at sample of Players and Teams tables

In [11]:
import pandas as pd
import sqlite3

# Load the data
teams_df = pd.read_csv('Teams.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
teams_df.to_sql('Teams', conn, index=False)

# Define the query
query = """
SELECT T1.team AS team1, T2.team AS team2, T1.goalsFor, T1.goalsAgainst
FROM Teams T1
JOIN Teams T2 ON T1.goalsFor = T2.goalsFor AND T1.goalsAgainst = T2.goalsAgainst
WHERE T1.team < T2.team;
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| team1     | team2        |   goalsFor |   goalsAgainst |
|:----------|:-------------|-----------:|---------------:|
| Italy     | Mexico       |          4 |              5 |
| England   | Nigeria      |          3 |              5 |
| England   | South Africa |          3 |              5 |
| Chile     | England      |          3 |              5 |
| Chile     | Nigeria      |          3 |              5 |
| Chile     | South Africa |          3 |              5 |
| Cameroon  | Greece       |          2 |              5 |
| Australia | Denmark      |          3 |              6 |
| Nigeria   | South Africa |          3 |              5 |


*1) Find all pairs of teams who have the same number of goalsFor as
each other and the same number of goalsAgainst as each other.
Return the teams and numbers of goalsFor and goalsAgainst.
Make sure to return each pair only once.*

In [12]:
import pandas as pd
import sqlite3

# Load the data
players_df = pd.read_csv('Players.csv')
teams_df = pd.read_csv('Teams.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
players_df.to_sql('Players', conn, index=False)
teams_df.to_sql('Teams', conn, index=False)

# Define the query
# We look for teams with ranking < 30 that are NOT in the list of teams 
# who have at least one player with > 150 passes.
query = """
SELECT team, ranking
FROM Teams
WHERE ranking < 30 
  AND team NOT IN (
    SELECT DISTINCT team 
    FROM Players 
    WHERE passes > 150
  );
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| team        |   ranking |
|:------------|----------:|
| France      |         9 |
| Nigeria     |        21 |
| Switzerland |        24 |


*2) Find all teams with ranking <30 where no player made more than 150 passes. Return the team and ranking.*

In [13]:
import pandas as pd
import sqlite3

# Load the data
players_df = pd.read_csv('Players.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
players_df.to_sql('Players', conn, index=False)

# Define the query
# Using a subquery to find the average of shots
query = """
SELECT surname, position, team
FROM Players
WHERE shots > 5 * (SELECT AVG(shots) FROM Players);
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| surname        | position   | team        |
|:---------------|:-----------|:------------|
| Higuain        | forward    | Argentina   |
| Messi          | forward    | Argentina   |
| Podolski       | forward    | Germany     |
| Boateng        | midfielder | Ghana       |
| Gyan           | forward    | Ghana       |
| Sneijder       | midfielder | Netherlands |
| van Persie     | forward    | Netherlands |
| Jong Tae-Se    | forward    | North Korea |
| Ronaldo        | forward    | Portugal    |
| Park Chu-Young | forward    | South Korea |
| Villa          | forward    | Spain       |
| Forlan         | forward    | Uruguay     |
| Suarez         | forward    | Uruguay     |
| Dempsey        | midfielder | USA         |


*3) Which players made more shots than 5x the overall average number of shots? Return the player surname, position, and team.*

In [14]:
import pandas as pd
import sqlite3

# Load the data
players_df = pd.read_csv('Players.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
players_df.to_sql('Players', conn, index=False)

# Define the query
query = """
SELECT surname, position, team
FROM Players
WHERE shots > 5 * (SELECT AVG(shots) FROM Players);
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| surname        | position   | team        |
|:---------------|:-----------|:------------|
| Higuain        | forward    | Argentina   |
| Messi          | forward    | Argentina   |
| Podolski       | forward    | Germany     |
| Boateng        | midfielder | Ghana       |
| Gyan           | forward    | Ghana       |
| Sneijder       | midfielder | Netherlands |
| van Persie     | forward    | Netherlands |
| Jong Tae-Se    | forward    | North Korea |
| Ronaldo        | forward    | Portugal    |
| Park Chu-Young | forward    | South Korea |
| Villa          | forward    | Spain       |
| Forlan         | forward    | Uruguay     |
| Suarez         | forward    | Uruguay     |
| Dempsey        | midfielder | USA         |


*4) Find all team-position pairs where the average number of passes made by players in that position on that team is greater than 150. Return the team-position pairs.*

In [15]:
import pandas as pd
import sqlite3

# Load the data
players_df = pd.read_csv('Players.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
players_df.to_sql('Players', conn, index=False)

# Define the query
query = """
SELECT team, position
FROM Players
GROUP BY team, position
HAVING AVG(passes) > 150;
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| team        | position   |
|:------------|:-----------|
| Argentina   | midfielder |
| Brazil      | defender   |
| Germany     | defender   |
| Germany     | midfielder |
| Ghana       | midfielder |
| Mexico      | defender   |
| Netherlands | defender   |
| Netherlands | midfielder |
| Spain       | defender   |
| Spain       | midfielder |


*5) Find all teams whose defenders averaged more than 150 passes. Return the team and average number of passes by defenders, in descending order of average passes.*

In [16]:
import pandas as pd
import sqlite3

# Load the data
players_df = pd.read_csv('Players.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
players_df.to_sql('Players', conn, index=False)

# Define the query
query = """
SELECT team, AVG(passes) AS avg_passes
FROM Players
WHERE position = 'defender'
GROUP BY team
HAVING avg_passes > 150
ORDER BY avg_passes DESC;
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| team        |   avg_passes |
|:------------|-------------:|
| Spain       |      213     |
| Brazil      |      190     |
| Germany     |      189.833 |
| Netherlands |      182.5   |
| Mexico      |      152.143 |


### <font color = 'green'>**Your Turn Extra - Advanced SQL on Titanic Data**</font>

<font color="red">File access required:</font> In Colab these extra problems require first uploading **Titanic.csv** using the *Files* feature in the left toolbar. If running the notebook on a local computer, simply ensure this file is in the same workspace as the notebook.

In [ ]:
# Load dataabase table from CSV file
with open('Titanic.csv') as f: Titanic = pd.read_csv(f, index_col=0)
%sql drop table if exists Titanic;
%sql --persist Titanic

#### Look at sample of Titanic table

In [17]:
import pandas as pd
import sqlite3

# Load the Titanic data
titanic_df = pd.read_csv('Titanic.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
titanic_df.to_sql('Titanic', conn, index=False)

# Define the query to find potential twin children
query = """
SELECT T1.last, T1.first AS first1, T2.first AS first2, T1.age, T1.embarked
FROM Titanic T1
JOIN Titanic T2 
  ON T1.last = T2.last 
  AND T1.age = T2.age 
  AND T1.embarked = T2.embarked
WHERE T1.age < 18 
  AND T1.first < T2.first;
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| last    | first1       | first2              |   age | embarked    |
|:--------|:-------------|:--------------------|------:|:------------|
| Baclini | Miss Eugenie | Miss Helene Barbara |  0.75 | Cherbourg   |
| Calic   | Mr. Jovo     | Mr. Petar           | 17    | Southampton |


*1) Find pairs of passengers who are likely to be twin children: same last name, same age, same embarkation, and age is under 18. Return each pair once, including their last name, first names, age, and embarkation city.*

In [18]:
import pandas as pd
import sqlite3

# Load the Titanic data
titanic_df = pd.read_csv('Titanic.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
titanic_df.to_sql('Titanic', conn, index=False)

# Define the query to find embarkation cities with > 40 passengers whose age is missing
query = """
SELECT embarked, COUNT(*) AS missing_age_count
FROM Titanic
WHERE age IS NULL
GROUP BY embarked
HAVING missing_age_count > 40;
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| embarked    |   missing_age_count |
|:------------|--------------------:|
| Queenstown  |                  49 |
| Southampton |                  90 |


*2) Which embarkation cities have more than 40 passengers whose age is missing? Reminder: Blanks in SQL tables are given a special value called 'null', and conditions 'A is null' and 'A is not null' can be used in Where clauses to check whether attribute A has the 'null' value.*

In [ ]:
import pandas as pd
import sqlite3

def setup_db():
    conn = sqlite3.connect(':memory:')
    files = {
        'Cities': 'Cities.csv',
        'Countries': 'Countries.csv',
        'Players': 'Players.csv',
        'Teams': 'Teams.csv',
        'Titanic': 'Titanic.csv'
    }
    for table, file in files.items():
        try:
            pd.read_csv(file).to_sql(table, conn, index=False, if_exists='replace')
        except:
            print(f"Warning: could not load {file}")
    return conn

conn = setup_db()

def run(query, title):
    print(f"\n--- {title} ---")
    try:
        res = pd.read_sql_query(query, conn)
        print(res.to_markdown(index=False))
    except Exception as e:
        print(f"Error: {e}")

*3) Find all classes where the average fare paid by passengers in that class was more than twice the overall average or less than half the overall average.*

In [19]:
import pandas as pd
import sqlite3

# Load the Titanic data
titanic_df = pd.read_csv('Titanic.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
titanic_df.to_sql('Titanic', conn, index=False)

# Define the query to find classes with extreme average fares
query = """
SELECT class
FROM Titanic
GROUP BY class
HAVING AVG(fare) > 2 * (SELECT AVG(fare) FROM Titanic)
    OR AVG(fare) < 0.5 * (SELECT AVG(fare) FROM Titanic);
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

|   class |
|--------:|
|       1 |
|       3 |


*4) What is the average number of passengers per last name? Hint: Requires using a subquery in the From clause*

In [20]:
import pandas as pd
import sqlite3

# Load the Titanic data
titanic_df = pd.read_csv('Titanic.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
titanic_df.to_sql('Titanic', conn, index=False)

# Define the query to find the average number of passengers per last name
query = """
SELECT AVG(name_count) AS avg_passengers_per_name
FROM (
    SELECT last, COUNT(*) AS name_count
    FROM Titanic
    GROUP BY last
);
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

|   avg_passengers_per_name |
|--------------------------:|
|                   1.33583 |


*5) List each class and its survival rate, i.e., the fraction of passengers in that class who survived. Hints: Use subqueries in the From clause to compute the number of survivers per class and total passengers per class, and force floating point division by multiplying one operand by 1.0*

In [21]:
import pandas as pd
import sqlite3

# Load the Titanic data
titanic_df = pd.read_csv('Titanic.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
titanic_df.to_sql('Titanic', conn, index=False)

# Define the query using subqueries in the FROM clause as requested
query = """
SELECT T.class, (S.survivors * 1.0 / T.total) AS survival_rate
FROM (
    SELECT class, COUNT(*) as total 
    FROM Titanic 
    GROUP BY class
) T
JOIN (
    SELECT class, COUNT(*) as survivors 
    FROM Titanic 
    WHERE survived = 'yes' 
    GROUP BY class
) S
ON T.class = S.class
ORDER BY T.class;
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

|   class |   survival_rate |
|--------:|----------------:|
|       1 |        0.62963  |
|       2 |        0.472826 |
|       3 |        0.242363 |


*6) Modify your previous query to return the survival rate by gender, i.e., of females and of males.*

In [22]:
import pandas as pd
import sqlite3

# Load the Titanic data
titanic_df = pd.read_csv('Titanic.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
titanic_df.to_sql('Titanic', conn, index=False)

# Define the query to find survival rate by gender
query = """
SELECT T.gender, (S.survivors * 1.0 / T.total) AS survival_rate
FROM (
    SELECT gender, COUNT(*) as total 
    FROM Titanic 
    GROUP BY gender
) T
JOIN (
    SELECT gender, COUNT(*) as survivors 
    FROM Titanic 
    WHERE survived = 'yes' 
    GROUP BY gender
) S
ON T.gender = S.gender;
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| gender   |   survival_rate |
|:---------|----------------:|
| F        |        0.742038 |
| M        |        0.188908 |


*7) Now return the survival rate of children versus adults, i.e., of passengers under age 18 versus those 18 or over (ignoring passengers whose age is missing).*

In [23]:
import pandas as pd
import sqlite3

# Load the Titanic data
titanic_df = pd.read_csv('Titanic.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
titanic_df.to_sql('Titanic', conn, index=False)

# Define the query to find survival rate by age group (child vs adult)
# Ignoring null ages
query = """
SELECT T.age_group, (S.survivors * 1.0 / T.total) AS survival_rate
FROM (
    SELECT 
        CASE WHEN age < 18 THEN 'child' ELSE 'adult' END AS age_group, 
        COUNT(*) as total 
    FROM Titanic 
    WHERE age IS NOT NULL
    GROUP BY age_group
) T
JOIN (
    SELECT 
        CASE WHEN age < 18 THEN 'child' ELSE 'adult' END AS age_group, 
        COUNT(*) as survivors 
    FROM Titanic 
    WHERE survived = 'yes' AND age IS NOT NULL
    GROUP BY age_group
) S
ON T.age_group = S.age_group;
"""

# Execute the query
result = pd.read_sql_query(query, conn)
print(result.to_markdown(index=False))

| age_group   |   survival_rate |
|:------------|----------------:|
| adult       |        0.381032 |
| child       |        0.539823 |


### <font color = 'green'>**Your Turn Extra - SQL Data Modification on Titanic Data**</font>

In [ ]:
# Reload table from CSV file
# NOTE: You may want to reload frequently to reset the data as you
# experiment with modifications
with open('Titanic.csv') as f: Titanic = pd.read_csv(f, index_col=0)
%sql drop table if exists Titanic;
%sql persist Titanic

*1) Subtract 5 from the fare paid by any passenger under the age of 10. Then compute the new average fare. NOTE: You can put multiple SQL statements in one cell separated by a semicolon.*

In [24]:
import pandas as pd
import sqlite3

# Load the Titanic data
titanic_df = pd.read_csv('Titanic.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
titanic_df.to_sql('Titanic', conn, index=False)

# Perform the update and compute the new average fare
cursor = conn.cursor()
cursor.execute("UPDATE Titanic SET fare = fare - 5 WHERE age < 10;")
conn.commit()

# Retrieve the new average fare
new_avg_fare = pd.read_sql_query("SELECT AVG(fare) FROM Titanic;", conn).iloc[0, 0]

# Save the transformed data to a new CSV file
transformed_df = pd.read_sql_query("SELECT * FROM Titanic", conn)
transformed_df.to_csv('Modified_Titanic.csv', index=False)

print(f"New Average Fare: {new_avg_fare}")

New Average Fare: 31.85709315375985


*2) Create a new table called Survivors, containing the last and first names of all passengers who survived. Then count the number of tuples in the new table.*

In [28]:
import sqlite3
import pandas as pd
import os

# 1. Setup paths
csv_file = 'Titanic.csv'
db_path = 'your_database.db'

# 2. Connect to the database
conn = sqlite3.connect(db_path)

try:
    # 3. Load Titanic.csv into the database if it's not already there
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
        df.to_sql('Titanic', conn, if_exists='replace', index=False)
        print(f"Successfully loaded {csv_file} into the 'Titanic' table.")
    else:
        print(f"Error: {csv_file} not found in the current directory.")

    # 4. Create the Survivors table
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS Survivors")
    cursor.execute("""
        CREATE TABLE Survivors AS 
        SELECT last, first 
        FROM Titanic 
        WHERE survived = 'yes';
    """)
    conn.commit()

    # 5. Count the results
    cursor.execute("SELECT COUNT(*) FROM Survivors")
    count = cursor.fetchone()[0]
    
    print(f"Number of tuples in Survivors table: {count}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    conn.close()

Successfully loaded Titanic.csv into the 'Titanic' table.
Number of tuples in Survivors table: 342


*3) In the Titanic table delete all but the highest-paying passengers.*

In [29]:
import sqlite3
import pandas as pd

# Connect to your database
conn = sqlite3.connect('your_database.db')
cursor = conn.cursor()

try:
    # 1. Perform the deletion
    # This deletes anyone whose fare is less than the maximum fare in the table
    cursor.execute("""
        DELETE FROM Titanic 
        WHERE fare < (SELECT MAX(fare) FROM Titanic);
    """)
    
    # 2. Commit the changes
    conn.commit()
    
    # 3. Retrieve the remaining passengers to verify
    remaining = pd.read_sql_query("SELECT * FROM Titanic", conn)
    
    # Save the updated table to a new CSV for your records
    remaining.to_csv('Highest_Paying_Passengers.csv', index=False)
    
    print(f"Deletion successful. {len(remaining)} passengers remain.")
    print(remaining)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    conn.close()

Deletion successful. 3 passengers remain.
      last                      first gender   age  class    fare   embarked  \
0  Cardeza  Mr. Thomas Drake Martinez      M  36.0      1  512.33  Cherbourg   
1  Lesurer              Mr. Gustave J      M  35.0      1  512.33  Cherbourg   
2     Ward                  Miss Anna      F  35.0      1  512.33  Cherbourg   

  survived  
0      yes  
1      yes  
2      yes  


*4) In what's left of the table after (3), insert a new tuple for yourself. You can decide your class, fare, where you embarked, and whether you survived. Then show the whole table.*

In [31]:
import sqlite3
import pandas as pd

# Connect to your database
conn = sqlite3.connect('your_database.db')
cursor = conn.cursor()

try:
    # 1. Insert your new record
    # We specify the columns to match your data
    cursor.execute("""
        INSERT INTO Titanic (last, first, class, age, gender, fare, embarked, survived)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, ('Cabanlig', 'Ruth Grace', 3, 23, 'F', 150, 'Philippines', 'yes'))
    
    # 2. Commit the change
    conn.commit()
    
    # 3. Show the whole table
    df_updated = pd.read_sql_query("SELECT * FROM Titanic", conn)
    
    print("Updated Titanic Table:")
    print(df_updated.to_markdown(index=False))

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    conn.close()

Updated Titanic Table:
| last     | first                     | gender   |   age |   class |   fare | embarked    | survived   |
|:---------|:--------------------------|:---------|------:|--------:|-------:|:------------|:-----------|
| Cardeza  | Mr. Thomas Drake Martinez | M        |    36 |       1 | 512.33 | Cherbourg   | yes        |
| Lesurer  | Mr. Gustave J             | M        |    35 |       1 | 512.33 | Cherbourg   | yes        |
| Ward     | Miss Anna                 | F        |    35 |       1 | 512.33 | Cherbourg   | yes        |
| Cabanlig | Ruth Grace                |          |   nan |       3 | 150    | Philippines | yes        |
| Cabanlig | Ruth Grace                | F        |    23 |       3 | 150    | Philippines | yes        |
